In [1]:
import pandas as pd
import numpy as np
import datetime
import statistics as stc
import matplotlib.pyplot as plt
import copy
import os
import math

from datetime import timedelta
from numpy import nan
from itertools import combinations
from matplotlib import pyplot as plt

# Carga de datos

In [2]:
ruta_un = "Datos_unificados/data_global.csv"
df_general = pd.read_csv(ruta_un)

df_general.drop(columns="Unnamed: 0", inplace=True)

#eliminamos las lineas con todas las columnas vacias 
df_general = df_general.dropna(how='all')

Se definen 2 tipos de datasets para pruebas:
 - Dataset donde se eliminan todas las lineas con algún valor vacío
 - Dataset donde se rellenan todas las lineas con algún valor vacío

In [3]:
df_general = df_general.drop(df_general[df_general["<# Operacions>"].isnull()].index)

df_general["<# Operacions>"] = df_general["<# Operacions>"].replace(0,1)

In [4]:
df_ligth = df_general[df_general["<Data Entrada Port>"].notna()]
df_ligth = df_ligth[df_ligth['<Data MAINGATE>'].notna()]
df_ligth = df_ligth[df_ligth['<Data SORTIDA>'].notna()]
df_ligth = df_ligth[df_ligth['<DataHora CODECO>'].notna()]

In [6]:
df_general.describe()

,<# Operacions>,<Minuts trànsit>
count,3.308074e+06,1.820054e+06
mean,1.395145e+00,3.636247e+01
std,5.159342e-01,3.999988e+01
min,1.000000e+00,8.330000e-02
25%,1.000000e+00,8.450000e+00
50%,1.000000e+00,2.221670e+01
75%,2.000000e+00,5.163330e+01
max,4.000000e+00,5.939000e+02


In [7]:
df_ligth.describe()

,<# Operacions>,<Minuts trànsit>
count,2.226761e+06,1.730067e+06
mean,1.467599e+00,3.607026e+01
std,5.296964e-01,3.961083e+01
min,1.000000e+00,8.330000e-02
25%,1.000000e+00,8.333300e+00
50%,1.000000e+00,2.200000e+01
75%,2.000000e+00,5.133330e+01
max,4.000000e+00,5.939000e+02


In [8]:
columnas = pd.Series(['<Data Operació>','<Visita>','<# Operacions>',
                             '<Contenidor>','<Ple/Buit>','<Tipus Operació>','<Data Entrada Port>',
                             '<Data MAINGATE>','<Data SORTIDA>','<DataHora CODECO>',
                             "<Lloc d'entrada port>",'<Minuts trànsit>'
                            ])

In [9]:
# % de valores vacíos para cada columna
def porc_val(df_general,columna):
    for i in columna:
        porcentaje = str(round(df_general[i].isnull().sum()/len(df_general)*100,2))
        print("Porcentaje de valores nan en la columna "+i+":")
        print("                                                        "+porcentaje + "%")
        print("")


In [10]:
porc_val(df_general,columnas)

Porcentaje de valores nan en la columna <Data Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Visita>:
                                                        0.0%

Porcentaje de valores nan en la columna <# Operacions>:
                                                        0.0%

Porcentaje de valores nan en la columna <Contenidor>:
                                                        0.0%

Porcentaje de valores nan en la columna <Ple/Buit>:
                                                        0.0%

Porcentaje de valores nan en la columna <Tipus Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data Entrada Port>:
                                                        27.11%

Porcentaje de valores nan en la columna <Data MAINGATE>:
                                                        14.15%

Porcentaje de valores nan en la columna <Data SORTIDA>:

Relaciones entre las columnas vacías de los valores de tiempos

# Rellenado de datos

In [11]:
df_general["<Data Entrada Port>"] = pd.to_datetime(df_general["<Data Entrada Port>"])
df_general["<Data MAINGATE>"] = pd.to_datetime(df_general["<Data MAINGATE>"])
df_general["<DataHora CODECO>"] = pd.to_datetime(df_general["<DataHora CODECO>"])
df_general["<Data SORTIDA>"] = pd.to_datetime(df_general["<Data SORTIDA>"])

In [12]:
#Elimino outlayers(columna minuts acces vacíos) para calcular tiempos medios

df_no_outlayer = df_general[np.invert(df_general["<Minuts trànsit>"].isnull())]

In [13]:
porc_val(df_ligth,columnas)

Porcentaje de valores nan en la columna <Data Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Visita>:
                                                        0.0%

Porcentaje de valores nan en la columna <# Operacions>:
                                                        0.0%

Porcentaje de valores nan en la columna <Contenidor>:
                                                        0.0%

Porcentaje de valores nan en la columna <Ple/Buit>:
                                                        0.0%

Porcentaje de valores nan en la columna <Tipus Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data Entrada Port>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data MAINGATE>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data SORTIDA>:
   

In [14]:
porc_val(df_general,columnas)

Porcentaje de valores nan en la columna <Data Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Visita>:
                                                        0.0%

Porcentaje de valores nan en la columna <# Operacions>:
                                                        0.0%

Porcentaje de valores nan en la columna <Contenidor>:
                                                        0.0%

Porcentaje de valores nan en la columna <Ple/Buit>:
                                                        0.0%

Porcentaje de valores nan en la columna <Tipus Operació>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data Entrada Port>:
                                                        27.11%

Porcentaje de valores nan en la columna <Data MAINGATE>:
                                                        14.15%

Porcentaje de valores nan en la columna <Data SORTIDA>:

# calculo de tiempos

In [15]:
#Calculo de minutos totales, desde que entra hasta que sale
tiempo_total = df_no_outlayer["<Data SORTIDA>"] - df_no_outlayer['<Data Entrada Port>']

#tiempo medio en general de los camiones cuando entran y salen
tiempo_medio_completo = tiempo_total.mean()

In [16]:
#Tiempo medio entre entrada y MAINGATE
tiempos_entradas = df_no_outlayer["<Data MAINGATE>"] - df_no_outlayer['<Data Entrada Port>']

#tiempo medio en general de los camiones cuando entran y salen
tiempo_medio_entrada = tiempos_entradas.mean()

In [19]:
tiempo_medio_entrada

Timedelta('0 days 00:36:21.748032750')

In [20]:
#Tiempo desde la entrada al muelle hasta la salida del puerto

df_no_outlayer.reset_index(drop=True, inplace=True)

tiempos_salida = df_no_outlayer['<Data SORTIDA>'] - df_no_outlayer["<DataHora CODECO>"] 

#tiempo medio en general de los camiones cuando entran y salen
tiempo_medio_salida = tiempos_salida.mean()

In [21]:
#se rectifican tiempos ilógicos

indice = tiempos_salida[tiempos_salida < pd.Timedelta("00:00:00")].index

df_no_outlayer['<Data SORTIDA>'].iloc[indice] = df_no_outlayer["<DataHora CODECO>"].iloc[indice]+tiempo_medio_salida

C:\Users\didac\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\didac\anaconda3\lib\site-packages\pandas\core\indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [22]:
#Tiempo medio para la columna Maingate

tiempos_maingate_izq = df_no_outlayer["<Data MAINGATE>"] - df_no_outlayer['<Data Entrada Port>']
tiempos_maingate_derch = df_no_outlayer["<DataHora CODECO>"] - df_no_outlayer["<Data MAINGATE>"]
tiempo_maingate_total = df_no_outlayer["<DataHora CODECO>"] - df_no_outlayer['<Data Entrada Port>'] 

#tiempo medio en general de los camiones cuando entran y salen
tiempo_medio_maingate_der = tiempos_maingate_derch.mean()
tiempo_medio_maingate_izq = tiempos_maingate_izq.mean()
tiempo_medio_total = tiempo_maingate_total.mean()

tiempo_total = tiempo_medio_maingate_der + tiempo_medio_maingate_izq

porc_maingate_derch = tiempo_medio_maingate_der.seconds / tiempo_total.seconds

porc_maingate_izq = tiempo_medio_maingate_izq.seconds / tiempo_total.seconds

In [23]:
#Tiempo medio para la columna DataHora CODECO

tiempos_codeco_izq = df_no_outlayer["<DataHora CODECO>"] - df_no_outlayer['<Data MAINGATE>']
tiempos_codeco_derch = df_no_outlayer["<Data SORTIDA>"] - df_no_outlayer["<DataHora CODECO>"]

#tiempo medio en general de los camiones cuando entran y salen
tiempo_medio_codeco_der = tiempos_codeco_derch.mean()
tiempo_medio_codeco_izq = tiempos_codeco_izq.mean()

tiempo_total = tiempo_medio_codeco_der + tiempo_medio_codeco_izq

porc_codeco_derch = tiempo_medio_codeco_der.seconds / tiempo_total.seconds

porc_codeco_izq = tiempo_medio_codeco_izq.seconds / tiempo_total.seconds

In [24]:
#tiempo medio entre Maingate y CODECO
tiempo_medio_maingate_der

Timedelta('0 days 00:25:01.974771110')

# Definición de máscaras

In [25]:
# tiempos_vacios de los extremos

lista_tiempos_vacios = df_general['<Data SORTIDA>'].isnull() & df_general['<Data Entrada Port>'].isnull() & df_general['<Data MAINGATE>'].isnull() & df_general['<DataHora CODECO>'].isnull()

entrada_vacio = df_general['<Data Entrada Port>'].isnull() & np.invert(df_general['<Data MAINGATE>'].isnull()) 
sortida_vacio = df_general['<Data SORTIDA>'].isnull() & np.invert(df_general['<DataHora CODECO>'].isnull())

In [26]:
# tiempos vacíos entre medio

mainagte_vacio = np.invert(df_general['<Data Entrada Port>'].isnull()) & df_general['<Data MAINGATE>'].isnull() & np.invert(df_general['<DataHora CODECO>'].isnull())
codeco_vacio = np.invert(df_general['<Data MAINGATE>'].isnull()) & df_general['<DataHora CODECO>'].isnull() & np.invert(df_general['<Data SORTIDA>'].isnull())

In [27]:
# tiempos vacíos en dos elementos

Entrada_maingate_vacio = df_general["<Data Entrada Port>"].isnull() & df_general["<Data MAINGATE>"].isnull() & np.invert(df_general["<Data SORTIDA>"].isnull()) & np.invert(df_general["<DataHora CODECO>"].isnull()) 
Maingate_codeco_vacio = np.invert(df_general['<Data Entrada Port>'].isnull()) & np.invert(df_general['<Data SORTIDA>'].isnull()) & df_general['<DataHora CODECO>'].isnull() & df_general['<Data MAINGATE>'].isnull() 
sortida_codeco_vacio = np.invert(df_general['<Data Entrada Port>'].isnull()) & np.invert(df_general['<Data MAINGATE>'].isnull()) & df_general['<DataHora CODECO>'].isnull() & df_general['<Data SORTIDA>'].isnull()

In [28]:
#mascaras necesarias

entrada_maingate_codeco_vacio = df_general["<Data Entrada Port>"].isnull() & df_general["<Data MAINGATE>"].isnull() & df_general["<DataHora CODECO>"].isnull() & np.invert(df_general["<Data SORTIDA>"].isnull()) 
salida_maingate_codeco_vacio = df_general["<Data SORTIDA>"].isnull() & df_general["<Data MAINGATE>"].isnull() & df_general["<DataHora CODECO>"].isnull() & np.invert(df_general["<Data Entrada Port>"].isnull()) 

# Rellenado de columans

Falta 1/4

Rellenado de tiempo de entrada

In [29]:
columnas = ['<Data Entrada Port>', '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']

for i in list(columnas):
    
    columnas = ['<Data Entrada Port>', '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']

    restantes = columnas
    restantes.remove(i)
    print("")
    print("solo columnas vacías: ")
    print(i)
    print((df_general[i].isnull() & np.invert(df_general[restantes[0]].isnull()) & np.invert(df_general[restantes[1]].isnull()) & np.invert(df_general[restantes[2]].isnull())).value_counts(dropna=False))


solo columnas vacías: 
<Data Entrada Port>
False    2847000
True      461074
dtype: int64

solo columnas vacías: 
<Data MAINGATE>
False    3255339
True       52735
dtype: int64

solo columnas vacías: 
<Data SORTIDA>
False    3296390
True       11684
dtype: int64

solo columnas vacías: 
<DataHora CODECO>
False    3220816
True       87258
dtype: int64


In [30]:
indice = df_general[entrada_vacio].index

df_entradas = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_entradas["<Data Entrada Port>"] = df_entradas["<Data MAINGATE>"] - tiempo_medio_entrada

df_general = pd.concat([df_general, df_entradas], axis=0)

df_general = df_general.sort_index()

Rellenado de tiempo de salida

In [31]:
indice = df_general[sortida_vacio].index

df_salidas = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_salidas["<Data SORTIDA>"] = df_salidas["<DataHora CODECO>"] + tiempo_medio_salida

df_general = pd.concat([df_general, df_salidas], axis=0)

df_general = df_general.sort_index()

Rellenado de tiempo de maingate

In [32]:
tiempos_completos = (df_general[mainagte_vacio]["<DataHora CODECO>"] - df_general[mainagte_vacio]["<Data Entrada Port>"])

tiempo_izq = tiempos_completos * porc_maingate_izq

indice = df_general[mainagte_vacio].index

df_salidas = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_salidas["<Data MAINGATE>"] = df_salidas["<Data Entrada Port>"] + tiempo_izq

df_general = pd.concat([df_general, df_salidas], axis=0)

df_general = df_general.sort_index()

Rellenado DataHora CODECO

In [33]:
tiempos_completos = (df_general[codeco_vacio]["<Data SORTIDA>"] - df_general[codeco_vacio]["<Data MAINGATE>"])

tiempo_izq = tiempos_completos * porc_codeco_izq

indice = df_general[codeco_vacio].index

df_salidas = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_salidas["<DataHora CODECO>"] = df_salidas["<Data MAINGATE>"] + tiempo_izq

df_general = pd.concat([df_general, df_salidas], axis=0)

df_general = df_general.sort_index()

Falta 2/4

In [34]:
# tiempos vacíos en dos elementos

Entrada_maingate_vacio = df_general["<Data Entrada Port>"].isnull() & df_general["<Data MAINGATE>"].isnull() & np.invert(df_general["<Data SORTIDA>"].isnull()) & np.invert(df_general["<DataHora CODECO>"].isnull()) 
Maingate_codeco_vacio = np.invert(df_general['<Data Entrada Port>'].isnull()) & np.invert(df_general['<Data SORTIDA>'].isnull()) & df_general['<DataHora CODECO>'].isnull() & df_general['<Data MAINGATE>'].isnull() 
sortida_codeco_vacio = np.invert(df_general['<Data Entrada Port>'].isnull()) & np.invert(df_general['<Data MAINGATE>'].isnull()) & df_general['<DataHora CODECO>'].isnull() & df_general['<Data SORTIDA>'].isnull() 

In [35]:
columnas = ['<Data Entrada Port>',
       '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']
       
temp = combinations(columnas, 2)
for i in list(temp):

  colum_temp= copy.copy(columnas)
  colum_temp.remove(i[0])
  colum_temp.remove(i[1])
  print("")
  print("solo columnas vacías: ")
  print(i[0],i[1])
  print((df_general[i[0]].isnull() & df_general[i[1]].isnull() & np.invert(df_general[colum_temp[0]].isnull()) & np.invert(df_general[colum_temp[1]].isnull())).value_counts(dropna=False))


solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE>
False    2909807
True      398267
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data SORTIDA>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data MAINGATE> <Data SORTIDA>
False    3308074
dtype: int64

solo columnas vacías: 
<Data MAINGATE> <DataHora CODECO>
False    3299908
True        8166
dtype: int64

solo columnas vacías: 
<Data SORTIDA> <DataHora CODECO>
False    3278982
True       29092
dtype: int64


Rellenado de Entrada y maingate

In [36]:
#Entrada y maingate vacíos
indice = df_general[Entrada_maingate_vacio].index

df_temporal = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_temporal["<Data MAINGATE>"] = df_temporal["<DataHora CODECO>"] - tiempos_codeco_izq.mean()
df_temporal["<Data Entrada Port>"] = df_temporal["<Data MAINGATE>"] - tiempo_medio_entrada

df_general = pd.concat([df_general, df_temporal], axis=0)

df_general = df_general.sort_index()

Rellenado de Codeco y sortida

In [37]:
indice = df_general[sortida_codeco_vacio].index

df_temporal = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_temporal["<DataHora CODECO>"] = df_temporal["<Data MAINGATE>"] + tiempos_maingate_derch.mean()
df_temporal["<Data SORTIDA>"] = df_temporal["<Data MAINGATE>"] + tiempos_codeco_derch

df_general = pd.concat([df_general, df_temporal], axis=0)

df_general = df_general.sort_index()

Rellenado Maingate y CODECO

In [39]:
#asignación de porcentajes
suma_tiempos = tiempos_entradas.mean() + tiempos_maingate_derch.mean() + tiempos_codeco_derch.mean()

porc_entradas = tiempos_entradas.mean()/ suma_tiempos
porc_espera = tiempos_maingate_derch.mean()/ suma_tiempos
porc_salida = tiempos_codeco_derch.mean()/ suma_tiempos


In [40]:
tiempos_completos = (df_general[Maingate_codeco_vacio]["<Data Entrada Port>"] - df_general[Maingate_codeco_vacio]["<Data SORTIDA>"])

entrada_main = tiempos_completos * porc_entradas
main_codeco = tiempos_completos * porc_espera

indice = df_general[Maingate_codeco_vacio].index

df_salidas = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_salidas["<Data MAINGATE>"] = df_salidas["<Data Entrada Port>"] + entrada_main
df_salidas["<DataHora CODECO>"] = df_salidas["<Data MAINGATE>"] + main_codeco

df_general = pd.concat([df_general, df_salidas], axis=0)

df_general = df_general.sort_index()

Faltan 3/4

In [41]:
#Comprobación de relleno completo

columnas = ['<Data Entrada Port>',
       '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']
       
temp = combinations(columnas, 3)
for i in list(temp):

  colum_temp= copy.copy(columnas)
  colum_temp.remove(i[0])
  colum_temp.remove(i[1])
  colum_temp.remove(i[2])
  print("")
  print("solo columnas vacías: ")
  print(i[0],i[1],i[2])

  print((df_general[i[0]].isnull() & df_general[i[1]].isnull() & df_general[i[2]].isnull() & np.invert(df_general[colum_temp[0]].isnull())).value_counts(dropna=False))


solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <Data SORTIDA>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <DataHora CODECO>
False    3305137
True        2937
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data SORTIDA> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data MAINGATE> <Data SORTIDA> <DataHora CODECO>
False    3307685
True         389
dtype: int64


In [42]:
indice = df_general[entrada_maingate_codeco_vacio].index

df_temporal = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_temporal["<DataHora CODECO>"] = df_temporal["<Data SORTIDA>"] - tiempos_salida.mean()
df_temporal["<Data MAINGATE>"] = df_temporal["<DataHora CODECO>"] - tiempos_maingate_derch.mean()
df_temporal["<Data Entrada Port>"] = df_temporal["<Data MAINGATE>"] - tiempos_maingate_izq.mean()

df_general = pd.concat([df_general, df_temporal], axis=0)

df_general = df_general.sort_index()

Relleno de columna codeco,maingate y sortida

In [43]:
indice = df_general[salida_maingate_codeco_vacio].index

df_temporal = df_general.loc[indice, :]

df_general = df_general.drop(indice, axis=0)

df_temporal["<Data MAINGATE>"] = df_temporal["<Data Entrada Port>"] + tiempos_maingate_izq.mean()
df_temporal["<DataHora CODECO>"] = df_temporal["<Data MAINGATE>"] + tiempos_maingate_derch.mean()
df_temporal["<Data SORTIDA>"] = df_temporal["<DataHora CODECO>"] + tiempos_salida.mean()

df_general = pd.concat([df_general, df_temporal], axis=0)

df_general = df_general.sort_index()

# Comprobación de columnas vacías

Para 2/4

In [44]:
columnas = ['<Data Entrada Port>',
       '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']
       
temp = combinations(columnas, 3)
for i in list(temp):

  colum_temp= copy.copy(columnas)
  colum_temp.remove(i[0])
  colum_temp.remove(i[1])
  colum_temp.remove(i[2])
  print("")
  print("solo columnas vacías: ")
  print(i[0],i[1],i[2])

  print((df_general[i[0]].isnull() & df_general[i[1]].isnull() & df_general[i[2]].isnull() & np.invert(df_general[colum_temp[0]].isnull())).value_counts(dropna=False))


solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <Data SORTIDA>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data SORTIDA> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data MAINGATE> <Data SORTIDA> <DataHora CODECO>
False    3308074
dtype: int64


Para 3/4

In [45]:
#Comprobación de relleno completo

columnas = ['<Data Entrada Port>',
       '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>']
       
temp = combinations(columnas, 3)
for i in list(temp):

  colum_temp= copy.copy(columnas)
  colum_temp.remove(i[0])
  colum_temp.remove(i[1])
  colum_temp.remove(i[2])
  print("")
  print("solo columnas vacías: ")
  print(i[0],i[1],i[2])

  print((df_general[i[0]].isnull() & df_general[i[1]].isnull() & df_general[i[2]].isnull() & np.invert(df_general[colum_temp[0]].isnull())).value_counts(dropna=False))


solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <Data SORTIDA>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data MAINGATE> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data Entrada Port> <Data SORTIDA> <DataHora CODECO>
False    3308074
dtype: int64

solo columnas vacías: 
<Data MAINGATE> <Data SORTIDA> <DataHora CODECO>
False    3308074
dtype: int64


In [46]:
# Se elimina las lineas donde los 4 tiempos están vacíos
len(set(df_general[df_general["<Data SORTIDA>"].isnull() & df_general["<Data MAINGATE>"].isnull() & df_general["<DataHora CODECO>"].isnull() & df_general["<Data Entrada Port>"].isnull()]["<Data Operació>"]))

indice = df_general[df_general["<Data SORTIDA>"].isnull() & df_general["<Data MAINGATE>"].isnull() & df_general["<DataHora CODECO>"].isnull() & df_general["<Data Entrada Port>"].isnull()].index

df_general = df_general.drop(indice)

indice = df_general[df_general["<Data SORTIDA>"].isnull()].index

df_general = df_general.drop(indice)

In [47]:
porc_val(df_general,columnas)

Porcentaje de valores nan en la columna <Data Entrada Port>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data MAINGATE>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data SORTIDA>:
                                                        0.0%

Porcentaje de valores nan en la columna <DataHora CODECO>:
                                                        0.0%



In [48]:
porc_val(df_ligth,columnas)

Porcentaje de valores nan en la columna <Data Entrada Port>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data MAINGATE>:
                                                        0.0%

Porcentaje de valores nan en la columna <Data SORTIDA>:
                                                        0.0%

Porcentaje de valores nan en la columna <DataHora CODECO>:
                                                        0.0%



In [49]:
df_general.info(null_counts=True)

<ipython-input-49-e3aed28ccf7a>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_general.info(null_counts=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 3290881 entries, 0 to 3308449
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   <Data Operació>        3290881 non-null  object        
 1   <Visita>               3290881 non-null  object        
 2   <# Operacions>         3290881 non-null  float64       
 3   <Contenidor>           3290881 non-null  object        
 4   <Ple/Buit>             3290881 non-null  object        
 5   <Tipus Operació>       3290881 non-null  object        
 6   <Data Entrada Port>    3290881 non-null  datetime64[ns]
 7   <Data MAINGATE>        3290881 non-null  datetime64[ns]
 8   <Data SORTIDA>         3290881 non-null  datetime64[ns]
 9   <DataHora CODECO>      3290881 non-null  datetime64[ns]
 10  <Lloc d'entrada port>  2400771 non-null  object        
 11  <Minuts trànsit>       1812178 non-null  float64       
dtypes: datetime64[ns](4), float6

In [50]:
df_ligth.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2226761 entries, 4203 to 3308293
Data columns (total 12 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   <Data Operació>        2226761 non-null  object 
 1   <Visita>               2226761 non-null  object 
 2   <# Operacions>         2226761 non-null  float64
 3   <Contenidor>           2226761 non-null  object 
 4   <Ple/Buit>             2226761 non-null  object 
 5   <Tipus Operació>       2226761 non-null  object 
 6   <Data Entrada Port>    2226761 non-null  object 
 7   <Data MAINGATE>        2226761 non-null  object 
 8   <Data SORTIDA>         2226761 non-null  object 
 9   <DataHora CODECO>      2226761 non-null  object 
 10  <Lloc d'entrada port>  2226761 non-null  object 
 11  <Minuts trànsit>       1730067 non-null  float64
dtypes: float64(2), object(10)
memory usage: 220.9+ MB


<ipython-input-50-519bb6802964>:1: FutureWarning: null_counts is deprecated. Use show_counts instead
  df_ligth.info(null_counts=True)


# Creación de la columna camiones

In [51]:
def desnidad_camiones(data):
  df_sort = pd.DataFrame(data["<# Operacions>"])
  df_sort["<Data SORTIDA>"] = data["<Data SORTIDA>"]
  df_sort["<Data SORTIDA>"] = pd.to_datetime(df_sort["<Data SORTIDA>"])
  df_sort.reset_index(drop=True, inplace=True)
    


  df_entr = pd.DataFrame(data["<# Operacions>"])
  df_entr["<Data Entrada Port>"] = data["<Data Entrada Port>"]
  df_entr["<Data Entrada Port>"] = pd.to_datetime(df_entr["<Data Entrada Port>"])
  df_entr.reset_index(drop=True, inplace=True)
    
  num_camiones = 0
  densidad = []
  index_ent = 0
  index_sal = 0

  while (len(df_sort)) > index_ent and  (len(df_entr)) > index_sal:

    entrada = comparativa_camiones(df_sort["<Data SORTIDA>"].iloc[index_sal],df_entr["<Data Entrada Port>"].iloc[index_ent])
    
    if entrada:
      num_camiones += 1/df_entr["<# Operacions>"].iloc[index_ent]
      index_ent += 1
      densidad.append(num_camiones)

    else:
      num_camiones -= 1/df_sort["<# Operacions>"].iloc[index_sal]
      index_sal += 1
        
    if index_ent%100000 == 0:
        print("quedan:",len(df_entr)- index_ent,"entradas")

  return densidad

In [52]:
def comparativa_camiones(salida,entrada):
#input de entradas y salidas actualizado y retornará si el mas próximo es una entrada
#o una salida. True -> entrada, False -> Salida

  if entrada < salida:
    entrada = True

  else :
    entrada = False
 # fuera se elimina la fecha correspondiente
  return entrada

In [53]:
num_camiones = desnidad_camiones(df_ligth)
num_camiones = list(np.around(np.array(num_camiones),0))

df_ligth["<num camiones>"] = num_camiones

quedan: 2126761 entradas
quedan: 2026761 entradas
quedan: 1926761 entradas
quedan: 1826761 entradas
quedan: 1726761 entradas
quedan: 1626761 entradas
quedan: 1526761 entradas
quedan: 1426761 entradas
quedan: 1326761 entradas
quedan: 1226761 entradas
quedan: 1126761 entradas
quedan: 1026761 entradas
quedan: 926761 entradas
quedan: 826761 entradas
quedan: 726761 entradas
quedan: 626761 entradas
quedan: 526761 entradas
quedan: 426761 entradas
quedan: 326761 entradas
quedan: 226761 entradas
quedan: 126761 entradas
quedan: 26761 entradas


In [54]:
num_camiones = desnidad_camiones(df_general)
num_camiones = list(np.around(np.array(num_camiones),0))

df_general["<num camiones>"] = num_camiones

quedan: 3190881 entradas
quedan: 3090881 entradas
quedan: 2990881 entradas
quedan: 2890881 entradas
quedan: 2790881 entradas
quedan: 2690881 entradas
quedan: 2590881 entradas
quedan: 2490881 entradas
quedan: 2390881 entradas
quedan: 2290881 entradas
quedan: 2190881 entradas
quedan: 2090881 entradas
quedan: 1990881 entradas
quedan: 1890881 entradas
quedan: 1790881 entradas
quedan: 1690881 entradas
quedan: 1590881 entradas
quedan: 1490881 entradas
quedan: 1390881 entradas
quedan: 1290881 entradas
quedan: 1190881 entradas
quedan: 1090881 entradas
quedan: 990881 entradas
quedan: 890881 entradas
quedan: 790881 entradas
quedan: 690881 entradas
quedan: 590881 entradas
quedan: 490881 entradas
quedan: 390881 entradas
quedan: 290881 entradas
quedan: 190881 entradas
quedan: 90881 entradas


Aquí tenemos 2 datasets:
- df_general: Todas las lineas iniciales, con los tiempos rellenados
- df_light: Con las lineas con algún tiempo vacío eliminado

La idea ahora, eliminar las lineas donde los minuts cua están vacíos

Se definen las categorias para los valores tanto objetivo como para las descriptivas

Podríamos eliminar la variable objetivo, y convertir la hora de salida como objetivo. Si no, dos columnas son las 

# Eliminación de outlayers y valores 0

In [55]:
df_general = df_general.dropna(subset=['<Minuts trànsit>'])

df_ligth = df_ligth.dropna(subset=['<Minuts trànsit>'])
df_ligth = df_ligth.dropna(subset=['<Minuts trànsit>'])

# Preparación de inputs

Las columnas de los inputs serán:

    - Numero de camiones
    - Minutos de transito anteriores
    - minutos desde CODECO a sortida
    - Mes de la data de operacio
    - Hora de entrada
    - Ple/buit
    - Tipo de operacion
    - Lugar de entrada

In [56]:
def selec_camion(df,operacion):
    
    df_temp = df[df["<# Operacions>"]==operacion]
    
    df_temp.reset_index(drop=True,inplace=True)
    
    indices = list(range(3,len(df_temp),operacion))
    
    return df_temp.iloc[indices]

In [57]:
df_ligth["<Data SORTIDA>"] =  pd.to_datetime(df_ligth["<Data SORTIDA>"])

df_ligth['<DataHora CODECO>'] = pd.to_datetime(df_ligth['<DataHora CODECO>'])

In [58]:
#se rectifican tiempos ilógicos

df_general.reset_index(drop=True, inplace=True)

indice = df_general[df_general["<Data SORTIDA>"] - df_general['<DataHora CODECO>'] < pd.Timedelta("00:00:00")].index

df_general['<Data SORTIDA>'].iloc[indice] = df_general["<DataHora CODECO>"].iloc[indice]+tiempo_medio_salida

In [59]:
#se rectifican tiempos ilógicos

df_ligth.reset_index(drop=True, inplace=True)

indice = df_ligth[df_ligth["<Data SORTIDA>"]- df_ligth['<DataHora CODECO>'] < pd.Timedelta("00:00:00")].index

df_ligth['<Data SORTIDA>'].iloc[indice] = df_ligth["<DataHora CODECO>"].iloc[indice]+tiempo_medio_salida

C:\Users\didac\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [60]:
df_cu = selec_camion(df_general,4)
df_tr = selec_camion(df_general,3)
df_do = selec_camion(df_general,2)
df_un = selec_camion(df_general,1)

df_general_trn = pd.concat([df_un,df_do,df_tr,df_cu])

In [61]:
tiempo_histo = [0,0,0,0,0,0,0,0,0,0]
       
for i in range(10,len(df_general_trn)):
    
    tiempo_histo.append(df_general_trn['<Minuts trànsit>'].iloc[i-10:i].mean())

    if i % 100000 == 0:
        print("quedan:", len(df_general)-i,"valores")
        
df_general_trn['<Hist. Minuts trànsit>'] = tiempo_histo

quedan: 1712178 valores
quedan: 1612178 valores
quedan: 1512178 valores
quedan: 1412178 valores
quedan: 1312178 valores
quedan: 1212178 valores
quedan: 1112178 valores
quedan: 1012178 valores
quedan: 912178 valores
quedan: 812178 valores
quedan: 712178 valores
quedan: 612178 valores
quedan: 512178 valores


# Revision de la creación del histórico codeco-sortida

In [63]:
tiempo_histo_co = [0,0,0,0,0,0,0,0,0,0]
       
for i in range(10,len(df_general_trn)):
    
    historic = df_general_trn["<Data SORTIDA>"].iloc[i-10:i]-df_general_trn['<DataHora CODECO>'].iloc[i-10:i]
    
    tiempo_histo_co.append(historic.mean())

    if i % 100000 == 0:
        print("quedan:", len(df_general)-i,"valores")
        
df_general_trn['<Hist. codeco sortida>'] = tiempo_histo_co

quedan: 1712178 valores
quedan: 1612178 valores
quedan: 1512178 valores
quedan: 1412178 valores
quedan: 1312178 valores
quedan: 1212178 valores
quedan: 1112178 valores
quedan: 1012178 valores
quedan: 912178 valores
quedan: 812178 valores
quedan: 712178 valores
quedan: 612178 valores
quedan: 512178 valores


In [64]:
#Eliminamos las 10 primeras lineas

df_general_trn.drop(df_general_trn[0:10].index, inplace=True)
df_general_trn.reset_index(drop=True,inplace=True)

In [65]:
df_general_trn['Mes'] = df_general_trn['<Data Entrada Port>'].dt.month

df_general_trn.columns

Index(['<Data Operació>', '<Visita>', '<# Operacions>', '<Contenidor>',
       '<Ple/Buit>', '<Tipus Operació>', '<Data Entrada Port>',
       '<Data MAINGATE>', '<Data SORTIDA>', '<DataHora CODECO>',
       '<Lloc d'entrada port>', '<Minuts trànsit>', '<num camiones>',
       '<Hist. Minuts trànsit>', '<Hist. codeco sortida>', 'Mes'],
      dtype='object')

In [66]:
df_cu = selec_camion(df_ligth,4)
df_tr = selec_camion(df_ligth,3)
df_do = selec_camion(df_ligth,2)
df_un = selec_camion(df_ligth,1)

df_general_trn_light = pd.concat([df_un,df_do,df_tr,df_cu])

In [67]:
df_general_trn_light["<Data SORTIDA>"] = pd.to_datetime(df_general_trn_light["<Data SORTIDA>"])
df_general_trn_light['<DataHora CODECO>'] = pd.to_datetime(df_general_trn_light['<DataHora CODECO>'])
df_general_trn_light['<Data Entrada Port>'] = pd.to_datetime(df_general_trn_light['<Data Entrada Port>'])

In [68]:
df_general_trn_light['Mes'] = df_general_trn_light['<Data Entrada Port>'].dt.month

In [69]:
tiempo_histo = [0,0,0,0,0,0,0,0,0,0]
       
for i in range(10,len(df_general_trn_light)):
    
    tiempo_histo.append(df_general_trn_light['<Minuts trànsit>'].iloc[i-10:i].mean())

    if i % 100000 == 0:
        print("quedan:", len(df_ligth)-i,"valores")
        
df_general_trn_light['<Hist. Minuts trànsit>'] = tiempo_histo

quedan: 1630067 valores
quedan: 1530067 valores
quedan: 1430067 valores
quedan: 1330067 valores
quedan: 1230067 valores
quedan: 1130067 valores
quedan: 1030067 valores
quedan: 930067 valores
quedan: 830067 valores
quedan: 730067 valores
quedan: 630067 valores
quedan: 530067 valores
quedan: 430067 valores


In [70]:
tiempo_histo_co = [0,0,0,0,0,0,0,0,0,0]
       
for i in range(10,len(df_general_trn_light)):
    
    historic = df_general_trn_light["<Data SORTIDA>"].iloc[i-10:i]-df_general_trn_light['<DataHora CODECO>'].iloc[i-10:i]
    
    tiempo_histo_co.append(historic.mean())

    if i % 100000 == 0:
        print("quedan:", len(df_ligth)-i,"valores")
        
df_general_trn_light['<Hist. codeco sortida>'] = tiempo_histo_co

quedan: 1630067 valores
quedan: 1530067 valores
quedan: 1430067 valores
quedan: 1330067 valores
quedan: 1230067 valores
quedan: 1130067 valores
quedan: 1030067 valores
quedan: 930067 valores
quedan: 830067 valores
quedan: 730067 valores
quedan: 630067 valores
quedan: 530067 valores
quedan: 430067 valores


In [71]:
#Eliminamos las 10 primeras lineas

df_general_trn_light.drop(df_general_trn_light[0:10].index, inplace=True)
df_general_trn_light.reset_index(drop=True,inplace=True)

# Categorización de los Datos


Categorización de número de camiones:
 - despejado
 - muy fluido
 - fluido
 - denso
 - muy denso
 

In [72]:
def categor(df_, columna, categorias):
    
    df = df_.copy()
    
    df =  df.sort_values(columna)
    df.reset_index(drop=True, inplace=True)
    
    part = math.floor(len(df)/len(categorias))
    
    indices_a = df.iloc[:part].index
    indices_b = df.iloc[part:(2*part)].index
    indices_c = df.iloc[(2*part):(3*part)].index
    indices_d = df.iloc[(3*part):(4*part)].index  
    indices_e = df.iloc[(4*part):].index 
    
    indices = [indices_a,indices_b,indices_c,indices_d,indices_e]
    
    print("los rango de valores son:")
    
    for i in indices:
        
        inicio = i[0]
        final= i[-1]
        
        print(df.iloc[inicio][columna],"-",df.iloc[final][columna],3)
        
    valores = []
    for j in range(len(categorias)):
        for i in range(len(indices_a)):
            n = 0
            if j != len(categorias)-1:
                valores.append(categorias[j])
    
    for i in range(len(indices_e)):
        valores.append(categorias[-1])
    
    df[columna] = valores
    
    return df
    

In [73]:
columna = "<num camiones>"

categorias = ["despejado","muy fluid","fluid","denso","muy denso"]

df_general_trn = categor(df_general_trn,columna,categorias)


freq = df_general_trn["<num camiones>"].value_counts() 
print(freq)

los rango de valores son:
0.0 - 148.0 3
148.0 - 349.0 3
349.0 - 596.0 3
596.0 - 913.0 3
913.0 - 2150.0 3
muy denso    279231
despejado    279229
denso        279229
muy fluid    279229
fluid        279229
Name: <num camiones>, dtype: int64


In [74]:
columna = "<num camiones>"

categorias = ["despejado","muy fluid","fluid","denso","muy denso"]

df_general_trn_light = categor(df_general_trn_light,columna,categorias)
#df_general_trn_light = df_general_trn_light.drop(["level_0"], axis=1)
#df_general_trn_light = df_general_trn_light.drop(["index"], axis=1)

freq = df_general_trn_light["<num camiones>"].value_counts() 
print(freq)

los rango de valores son:
0.0 - 114.0 3
114.0 - 270.0 3
270.0 - 464.0 3
464.0 - 711.0 3
711.0 - 1448.0 3
muy denso    265676
despejado    265674
denso        265674
muy fluid    265674
fluid        265674
Name: <num camiones>, dtype: int64


In [75]:
def meses(df):
    
    meses = ["ENERO","FEBRERO","MARZO","ABRIL","MAYO","JUNIO","JULIO","AGOSTO","SEPTIEMBRE","OCTUBRE","NOVIEMBRE","DICIEMBRE"]
    
    df.loc[df['Mes'] == 1, 'Mes'] = meses[0]
    df.loc[df['Mes'] == 2, 'Mes'] = meses[1]
    df.loc[df['Mes'] == 3, 'Mes'] = meses[2]
    df.loc[df['Mes'] == 4, 'Mes'] = meses[3]
    df.loc[df['Mes'] == 5, 'Mes'] = meses[4]
    df.loc[df['Mes'] == 6, 'Mes'] = meses[5]
    df.loc[df['Mes'] == 7, 'Mes'] = meses[6]
    df.loc[df['Mes'] == 8, 'Mes'] = meses[7]
    df.loc[df['Mes'] == 9, 'Mes'] = meses[8]
    df.loc[df['Mes'] == 10, 'Mes'] = meses[9]
    df.loc[df['Mes'] == 11, 'Mes'] = meses[10]
    df.loc[df['Mes'] == 12, 'Mes'] = meses[11]
    
    return df

In [76]:
df_general_trn = meses(df_general_trn)
df_general_trn_light = meses(df_general_trn_light)

In [77]:
df_general_trn = categor(df_general_trn, "<Minuts trànsit>", categorias)

df_general_trn_light = categor(df_general_trn_light, "<Minuts trànsit>", categorias)

los rango de valores son:
0.0833 - 6.483300000000001 3
6.483300000000001 - 14.6 3
14.6 - 29.9167 3
29.9167 - 57.7167 3
57.7167 - 593.9 3
los rango de valores son:
0.0833 - 6.4 3
6.4 - 14.4167 3
14.4167 - 29.6 3
29.6 - 57.2333 3
57.2333 - 593.9 3


In [78]:
df_general_trn = categor(df_general_trn, "<Hist. Minuts trànsit>", categorias)

df_general_trn_light = categor(df_general_trn_light, "<Hist. Minuts trànsit>", categorias)

los rango de valores son:
1.8616799999999998 - 13.10666 3
13.10666 - 21.490009999999998 3
21.490009999999998 - 33.13833 3
33.13833 - 49.80834 3
49.80834 - 481.47166000000004 3
los rango de valores son:
1.8616799999999998 - 12.995 3
12.995 - 21.31167 3
21.31167 - 32.888329999999996 3
32.888329999999996 - 49.36833 3
49.36833 - 481.47166000000004 3


In [79]:
df_general_trn = categor(df_general_trn, "<Hist. codeco sortida>", categorias)

df_general_trn_light = categor(df_general_trn_light, "<Hist. codeco sortida>", categorias)

los rango de valores son:
0 days 00:00:08.500000 - 0 days 00:03:10.578487229 3
0 days 00:03:10.579715127 - 0 days 00:04:23.700000 3
0 days 00:04:23.700000 - 0 days 00:06:03.237582231 3
0 days 00:06:03.237622789 - 0 days 00:11:21.500000 3
0 days 00:11:21.500000 - 0 days 03:45:37.300000 3
los rango de valores son:
0 days 00:00:08.500000 - 0 days 00:02:54.200000 3
0 days 00:02:54.200000 - 0 days 00:04:01.400000 3
0 days 00:04:01.400000 - 0 days 00:05:24.300000 3
0 days 00:05:24.300000 - 0 days 00:07:10.200000 3
0 days 00:07:10.200000 - 0 days 03:45:37.300000 3


In [80]:
columnas_eliminar = ['<Data Operació>', '<Visita>', '<# Operacions>', '<Contenidor>','<Data Entrada Port>', '<Data MAINGATE>', '<Data SORTIDA>','<DataHora CODECO>']

In [81]:
df_general_trn = df_general_trn.drop(columnas_eliminar, axis=1)

In [82]:
df_general_trn_light = df_general_trn_light.drop(columnas_eliminar, axis=1)

In [83]:
df_general_trn_light.columns

Index(['<Ple/Buit>', '<Tipus Operació>', '<Lloc d'entrada port>',
       '<Minuts trànsit>', '<num camiones>', 'Mes', '<Hist. Minuts trànsit>',
       '<Hist. codeco sortida>'],
      dtype='object')

In [84]:
mask1 = df_general_trn["<Minuts trànsit>"]== "muy denso"

mask2 = df_general_trn["<Hist. Minuts trànsit>"]== "despejado"

In [85]:
len(df_general_trn[mask1&mask2])

1771

Fijando el valor de muy denso en Minutos de tránsito, miramos los valores del historico de miuntos de transito

muy densa 160604
denso      78959
fluid      30780
muy fluido  7117
despejado   1771


In [86]:
mask3 = df_general_trn["<num camiones>"]== "muy denso"

mask4 = df_general_trn["<Hist. Minuts trànsit>"]== "despejado"

In [87]:
len(df_general_trn[mask3&mask4])

29554

In [88]:
df_general_trn["<Hist. Minuts trànsit>"].unique()

array(['fluid', 'denso', 'muy fluid', 'muy denso', 'despejado'],
      dtype=object)

Fijando el valor de muy denso en num camiones, miramos los valores del historico de miuntos de transito

muy densa  77470
denso      68518
fluid      58129 
muy fluido 45560 
despejado  29554

In [89]:
len(df_general_trn)

1396147

In [90]:
len(df_general_trn_light)

1328372

# Guardar los datasets destinados a modelo

In [ ]:
#Ruta de guardado

dir_out = os.makedirs('Datos_categorizados/', exist_ok=True)

df_general_trn.to_csv('Datos_categorizados/df_general_trn.csv')

df_general_trn_light.to_csv('Datos_categorizados/df_general_trn_light.csv')